In [1]:
from langchain.chat_models import AzureChatOpenAI
from dotenv import load_dotenv
import os
import openai
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
import pandas as pd
from tqdm import tqdm


In [2]:
LLM_MODEL = 'gpt-4'


In [3]:

load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
API_KEY = os.environ.get('API_KEY')
API_VERSION = os.environ.get('API_VERSION')
RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = API_VERSION
chat_deployment_id = LLM_MODEL
chat_model_id = chat_deployment_id
temperature = 0
chat_model = AzureChatOpenAI(openai_api_key=API_KEY, openai_api_version=API_VERSION, azure_deployment=chat_deployment_id, azure_endpoint=RESOURCE_ENDPOINT, temperature=temperature)
load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
username = os.environ.get('NEO4J_USER')
password = os.environ.get('NEO4J_PSW')
url = os.environ.get('NEO4J_URL')
database = os.environ.get('NEO4J_DB')



/Users/karthiksoman/anaconda3/envs/cypher_rag/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [4]:
graph = Neo4jGraph(url=url, username=username, password=password, database = database)
chain = GraphCypherQAChain.from_llm(chat_model, graph=graph, verbose=True, validate_cypher=True,return_intermediate_steps=True)



In [5]:
question = "What are the genes associated with hypochondrogenesis?"

cypher_rag_answer = chain.invoke({"query":question})




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "hypochondrogenesis"})-[:ASSOCIATES_DaG]->(g:Gene) RETURN g.name
Full Context:
[{'g.name': 'COL2A1'}]

> Finished chain.


In [6]:
cypher_rag_answer

{'query': 'What are the genes associated with hypochondrogenesis?',
 'result': 'The gene associated with hypochondrogenesis is COL2A1.',
 'intermediate_steps': [{'query': 'MATCH (d:Disease {name: "hypochondrogenesis"})-[:ASSOCIATES_DaG]->(g:Gene) RETURN g.name'},
  {'context': [{'g.name': 'COL2A1'}]}]}

In [7]:
cypher_rag_answer["intermediate_steps"]

[{'query': 'MATCH (d:Disease {name: "hypochondrogenesis"})-[:ASSOCIATES_DaG]->(g:Gene) RETURN g.name'},
 {'context': [{'g.name': 'COL2A1'}]}]